# HurdleDMR from R

HurdleDMR.jl is a Julia implementation of the Hurdle Distributed Multinomial Regression (HDMR), as described in:

Kelly, Bryan, Asaf Manela, and Alan Moreira (2021). Text Selection. [Working paper](http://apps.olin.wustl.edu/faculty/manela/kmm/textselection/).

It includes a Julia implementation of the Distributed Multinomial Regression (DMR) model of [Taddy (2015)](https://arxiv.org/abs/1311.6139).

This tutorial explains how to use this package from R via the JuliaCall package that is available on CRAN.

## Setup

### Install Julia

First, install Julia itself. The easiest way to do that is to get the latest stable release from the official [download page](https://julialang.org/downloads/). An alternative is to install [JuliaPro](https://juliacomputing.com/products/juliapro/).

Once installed, open julia in a terminal, press ] to activate package manager and add the following packages:
```julia
    pkg> add RCall HurdleDMR GLM Lasso
```

### The JuliaCall package for R

Now, back to R

In [1]:
install.packages("JuliaCall")

Installing package into ‘/home/user/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)



Load the JuliaCall library and setup julia

In [2]:
library(JuliaCall)
jl <- julia_setup(installJulia = TRUE)

Julia version 1.5.3 at location /home/user/packages/julias/julia-1.5.3/bin will be used.

Loading setup script for JuliaCall...

Finish loading setup script for JuliaCall.



`jl` allows us to evaluate julia code.

### Example Data

The data should either be an n-by-p covars matrix or a DataFrame containing the covariates, and a (sparse) n-by-d counts matrix.

For illustration we'll analyse the State of the Union text that is roughly annual and relate it to stock market returns.

The `sotu.jl` script compiles stock market execss returns and the State of the Union Address texts into a matching DataFrame `covarsdf` and a sparse document-term matrix `counts`.

In [3]:
julia_install_package_if_needed("HurdleDMR")
julia_install_package_if_needed("Lasso")
julia_install_package_if_needed("CSV")
julia_install_package_if_needed("DataDeps")
julia_install_package_if_needed("GLM")
julia_install_package_if_needed("DataFrames")
julia_install_package_if_needed("FamaFrenchData")
julia_install_package_if_needed("TextAnalysis")
julia_install_package_if_needed("SparseArrays")
julia_install_package_if_needed("PyCall")

In [4]:
# # if the above doesn't work, you can also try
# jl$command("import Pkg")
# jl$command("Pkg.add([\"Lasso\", \"CSV\", \"DataDeps\", \"DataFrames\", \"Pandas\", \"FamaFrenchData\", \"TextAnalysis\", \"SparseArrays\", \"PyCall\"])")

In [5]:
jl$call("include", "sotu.jl")

[[1]]
   Date    Rem             President
1  1927  29.47       Calvin Coolidge
2  1928  35.39       Calvin Coolidge
3  1929 -19.54        Herbert Hoover
4  1930 -31.23        Herbert Hoover
5  1931 -45.11        Herbert Hoover
6  1932  -9.39        Herbert Hoover
7  1934   3.02 Franklin D. Roosevelt
8  1935  44.96 Franklin D. Roosevelt
9  1936  32.07 Franklin D. Roosevelt
10 1937 -34.96 Franklin D. Roosevelt
11 1938  28.48 Franklin D. Roosevelt
12 1939   2.70 Franklin D. Roosevelt
13 1940  -7.14 Franklin D. Roosevelt
14 1941 -10.53 Franklin D. Roosevelt
15 1942  16.20 Franklin D. Roosevelt
16 1943  27.96 Franklin D. Roosevelt
17 1944  20.97 Franklin D. Roosevelt
18 1945  38.38 Franklin D. Roosevelt
19 1946  -6.73       Harry S. Truman
20 1947   2.95       Harry S. Truman
21 1948   1.07       Harry S. Truman
22 1949  19.12       Harry S. Truman
23 1950  28.82       Harry S. Truman
24 1951  19.22       Harry S. Truman
25 1952  11.80       Harry S. Truman
26 1953  -1.05  Dwight D. Eisenh

In [6]:
library(Matrix)
rcounts <- as(jl$eval("counts"), "sparseMatrix")
rcovarsdf <- jl$eval("covarsdf")

In [7]:
rcounts

92 x 163 sparse Matrix of class "dgCMatrix"
                                                                              
 [1,]  1  . . . . .  . .  . 1 . .  . . . . . . . .  . . . . . . 1 . . 1 .  . .
 [2,]  .  . . . . .  1 .  . 2 . .  . . . . . . . 3  . . . . . . 1 1 . . .  2 .
 [3,]  .  . . . 1 .  2 .  . 3 . .  . . 1 . . . . 4  . . . . 1 . . . . 1 .  1 .
 [4,]  .  . . . . .  . .  . 1 . .  1 . 2 . . . . 1  . . . . . . . . 1 . .  3 .
 [5,]  .  . . . . .  . .  . 1 . .  . . 1 . . . . .  . . . . . . . . . . .  . .
 [6,]  .  . . . . .  . .  . . . 1  . . . . . . . .  . . . . . . . . 1 . .  . .
 [7,]  .  . . . . .  3 .  . . . .  . . . . . . . .  . . . . . . . . . . .  . .
 [8,]  .  . . . . .  2 .  . . . .  . 1 1 . . . . 1  . . . . 1 . . . . 1 .  . .
 [9,]  .  . . . . .  . .  1 . . 1  . . . . . . . .  . . . . . . . . . . .  . .
[10,]  .  . . . . .  . .  . . . .  . . . . . . . .  . . . 2 . . . . . . .  . .
[11,]  .  . . . . .  1 .  . . . 2  4 . . . . . . .  . . . 2 . . . . . . .  . .
[12,]  .

In [8]:
rcovarsdf

,Date,Rem,President
,<int>,<dbl>,<chr>
1,1927,29.47,Calvin Coolidge
2,1928,35.39,Calvin Coolidge
3,1929,-19.54,Herbert Hoover
4,1930,-31.23,Herbert Hoover
5,1931,-45.11,Herbert Hoover
6,1932,-9.39,Herbert Hoover
7,1934,3.02,Franklin D. Roosevelt
8,1935,44.96,Franklin D. Roosevelt
9,1936,32.07,Franklin D. Roosevelt


### Add parallel workers and make HurdleDMR package available to workers

In [9]:
jl$command("using Distributed")
jl$command("addprocs(4)")

julia_library("HurdleDMR")
jl$command("@everywhere using HurdleDMR")

4-element Array{Int64,1}:
 2
 3
 4
 5

First we need to convert the R sparseMatrix to julia. We do this in pieces because if we materialize the entire dense matrix representation it may require more memory than we have.

In [10]:
A <- as(rcounts, "TsparseMatrix")
jl$assign("i", A@i + 1)
jl$assign("j", A@j + 1)
jl$assign("v", A@x)
jl$assign("m", dim(A)[1])
jl$assign("n", dim(A)[2])
jl$command("jcounts = sparse(i, j, v, m, n);")
# Note: jcounts is now a julia object accessible to julia not R

# # uncomment the following line to make sure the matrices are the same (DO NOT TRY WITH BIG DATA)
# all(jl$eval("jcounts") == rcounts)

## Distributed Multinomial Regression (DMR)

The Distributed Multinomial Regression (DMR) model of Taddy (2015) is a highly scalable
approximation to the Multinomial using distributed (independent, parallel)
Poisson regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covarsdf`.

To fit a DMR:

In [11]:
jl$assign("jcovars", as.matrix(rcovarsdf[['Rem']]))
jl$command("m = dmr(jcovars, jcounts)")

DMRCoefs{Array{Float64,2},MinAICc}([-5.433977770164587 -5.169034226440705 … -5.37899826953734 -4.430372688887225; -0.03460506144624849 -0.017470711240303454 … 0.007352225999870346 0.004405577373050878], true, 92, 163, 1, MinAICc(2))

or with a dataframe and formula, by first converting the pandas dataframe to julia

In [12]:
jl$assign("jcovarsdf", rcovarsdf) # assign to julia dataframe
jl$command("mf = @model(c ~ Rem)") # model formula
jl$command("m = fit(DMR, mf, jcovarsdf, jcounts)")

1-part model: [c ~ Rem]

TableCountsRegressionModel{DMRCoefs{Array{Float64,2},MinAICc},DataFrame,SparseMatrixCSC{Float64,Int64}}

1-part model: [c ~ Rem]

We can get the coefficients matrix for each variable + intercept as usual with

In [13]:
jl$eval("coef(m)")

-5.43397777,-5.16903423,-5.483462,-5.87732380,-5.873792642,-5.771144,-3.235432812,-5.38168,-4.410759806,-5.678937911,⋯,-5.232148,-5.829416147,-5.289306,-5.76087053,-5.414535e+00,-4.72731232,-5.64625805,-4.924965597,-5.378998270,-4.430372689
-0.03460506,-0.01747071,0.000000,0.01188229,0.006711126,0.000000,0.007545783,0.00000,0.009774115,-0.008737706,⋯,0.000000,0.007185718,0.000000,-0.02318809,9.579289e-06,-0.01967833,0.00122726,0.007426632,0.007352226,0.004405577


By default we only return the AICc maximizing coefficients.
To also get back the entire regulatrization paths, run

In [14]:
jl$command("paths = dmrpaths(jcovars, jcounts)")

DMRPaths(Union{Missing, GammaLassoPath}[Poisson GammaLassoPath (44) solutions for 2 predictors in 196 iterations):
─────────────────────────────────────
 λ pct_dev ncoefs
─────────────────────────────────────
 [1] 0.209459 3.76255e-13 1
 [2] 0.190851 0.0225008 1
 [3] 0.173897 0.0410081 1
 [4] 0.158448 0.0562654 1
 [5] 0.144372 0.0688665 1
 [6] 0.131546 0.0792886 1
 [7] 0.11986 0.0879183 1
 [8] 0.109212 0.0950703 1
 [9] 0.09951 0.101002 1
[10] 0.0906698 0.105923 1
[11] 0.082615 0.110009 1
[12] 0.0752757 0.113401 1
[13] 0.0685884 0.116218 1
[14] 0.0624952 0.118559 1
[15] 0.0569433 0.120503 1
[16] 0.0518846 0.122119 1
[17] 0.0472753 0.123461 1
[18] 0.0430755 0.124576 1
[19] 0.0392488 0.125503 1
[20] 0.035762 0.126273 1
[21] 0.032585 0.126913 1
[22] 0.0296903 0.127444 1
[23] 0.0270527 0.127886 1
[24] 0.0246494 0.128253 1
[25] 0.0224596 0.128557 1
[26] 0.0204644 0.128811 1
[27] 0.0186464 0.129021 1
[28] 0.0169899 0.129196 1
[29] 0.0154805 0.129341 1
[30] 0.0141053 0.129461 1
[31] 0.0128522 0.129562 1
[32] 0.0117105 0.129645 1
[33] 0.0106701 0.129714 1
[34] 0.00972223 0.129771 1
[35] 0.00885853 0.129819 1
[36] 0.00807156 0.129858 1
[37] 0.00735451 0.129891 1
[38] 0.00670115 0.129918 1
[39] 0.00610584 0.129941 1
[40] 0.00556342 0.12996 1
[41] 0.00506918 0.129976 1
[42] 0.00461885 0.129989 1
[43] 0.00420852 0.129999 1
[44] 0.00383465 0.130008 1
─────────────────────────────────────, Poisson GammaLassoPath (35) solutions for 2 predictors in 156 iterations):
────────────────────────────────────
 λ pct_dev ncoefs
────────────────────────────────────
 [1] 0.133903 9.4369e-15 0
 [2] 0.122007 0.00420333 1
 [3] 0.111169 0.00767482 1
 [4] 0.101293 0.0105443 1
 [5] 0.0922941 0.012918 1
 [6] 0.084095 0.0148826 1
 [7] 0.0766242 0.0165094 1
 [8] 0.0698171 0.0178571 1
 [9] 0.0636148 0.018974 1
[10] 0.0579634 0.0198998 1
[11] 0.0528141 0.0206674 1
[12] 0.0481222 0.021304 1
[13] 0.0438472 0.021832 1
[14] 0.0399519 0.02227 1
[15] 0.0364027 0.0226335 1
[16] 0.0331688 0.022935 1
[17] 0.0302222 0.0231852 1
[18] 0.0275373 0.0233928 1
[19] 0.025091 0.0235652 1
[20] 0.022862 0.0237082 1
[21] 0.020831 0.0238269 1
[22] 0.0189804 0.0239254 1
[23] 0.0172942 0.0240072 1
[24] 0.0157579 0.0240751 1
[25] 0.014358 0.0241315 1
[26] 0.0130825 0.0241783 1
[27] 0.0119202 0.0242171 1
[28] 0.0108613 0.0242493 1
[29] 0.0098964 0.0242761 1
[30] 0.00901723 0.0242983 1
[31] 0.00821616 0.0243167 1
[32] 0.00748626 0.0243321 1
[33] 0.0068212 0.0243448 1
[34] 0.00621523 0.0243553 1
[35] 0.00566308 0.0243641 1
────────────────────────────────────, Poisson GammaLassoPath (33) solutions for 2 predictors in 142 iterations):
─────────────────────────────────────
 λ pct_dev ncoefs
─────────────────────────────────────
 [1] 0.0633431 2.22045e-16 0
 [2] 0.0577159 0.00289073 1
 [3] 0.0525885 0.00528278 1
 [4] 0.0479167 0.00726305 1
 [5] 0.0436599 0.00890304 1
 [6] 0.0397813 0.0102617 1
 [7] 0.0362472 0.0113875 1
 [8] 0.0330271 0.0123206 1
 [9] 0.0300931 0.0130942 1
[10] 0.0274197 0.0137356 1
[11] 0.0249838 0.0142676 1
[12] 0.0227643 0.0147088 1
[13] 0.020742 0.0150747 1
[14] 0.0188993 0.0153783 1
[15] 0.0172204 0.0156302 1
[16] 0.0156906 0.0158392 1
[17] 0.0142967 0.0160126 1
[18] 0.0130266 0.0161565 1
[19] 0.0118693 0.0162759 1
[20] 0.0108149 0.016375 1
[21] 0.00985413 0.0164573 1
[22] 0.00897872 0.0165255 1
[23] 0.00818107 0.0165822 1
[24] 0.00745429 0.0166292 1
[25] 0.00679207 0.0166682 1
[26] 0.00618868 0.0167006 1
[27] 0.0056389 0.0167275 1
[28] 0.00513795 0.0167498 1
[29] 0.00468151 0.0167684 1
[30] 0.00426562 0.0167838 1
[31] 0.00388667 0.0167965 1
[32] 0.00354139 0.0168071 1
[33] 0.00322678 0.0168159 1
─────────────────────────────────────, Poisson GammaLassoPath (33) solutions for 2 predictors in 133 iterations):
────────────────────────────────────
 λ pct_dev ncoefs
────────────────────────────────────
 [1] 0.0511147 2.9853e-9 1
 [2] 0.0465738 0.00299059 1
 [3] 0.0424363 0.00548318 1
 [4] 0.0386664 0.00756034 1
 [5] 0.0352314 0.00929098 1
 [6] 0.0321015 0.0107326 1
 [7] 0.02924

We can now select, for example the coefficients that minimize 10-fold CV mse (takes a little longer)

In [16]:
jl$command("using Lasso: MinCVmse;")
jl$command("segselect = MinCVKfold{MinCVmse}(10);")
jl$eval("coef(paths, segselect)")

-5.448371e+00,-5.232148,-5.43167163,-5.771144e+00,-5.819935e+00,-5.771144,-3.173760e+00,-5.38168,-4.327030e+00,-5.724624e+00,⋯,-5.232148,-5.771144e+00,-5.289306,-5.76087053,-5.414469e+00,-4.72731232,-5.637613e+00,-4.924965597,-5.319159e+00,-4.396826e+00
-5.117028e-14,0.000000,-0.01060117,1.036714e-09,2.086696e-08,0.000000,1.182784e-14,0.00000,2.361194e-09,-9.887320e-13,⋯,0.000000,8.493204e-12,0.000000,-0.02318809,6.425219e-09,-0.01967833,2.782913e-09,0.007426632,7.622884e-10,7.830152e-10


## Hurdle Distributed Multinomial Regression (HDMR)

For highly sparse counts, as is often the case with text that is selected for
various reasons, the Hurdle Distributed Multinomial Regression (HDMR) model of
Kelly, Manela, and Moreira (2021), may be superior to the DMR. It approximates
a higher dispersion Multinomial using distributed (independent, parallel)
Hurdle regressions, one for each of the d categories (columns) of a large `counts` matrix,
on the `covars`. It allows a potentially different sets of covariates to explain
category inclusion ($h=1{c>0}$), and repetition ($c>0$) using the optional `inpos` and `inzero` keyword arguments.

Both the model for zeroes and for positive counts are regularized by default,
using `GammaLassoPath`, picking the AICc optimal segment of the regularization
path.

HDMR can be fitted:

In [17]:
jl$assign("jcovars", as.matrix(rcovarsdf[['Rem']]))
jl$command("m = hdmr(jcovars, jcounts)")

HDMRCoefs{InclusionRepetition,Array{Float64,2},MinAICc,UnitRange{Int64}}([-4.10095407845034 -2.463114021079464 … -4.9712845243783885 -3.9926360951822044; -0.02713335949136956 -0.03288064802519407 … 0.016349965746303912 0.025092391911020286], [-0.5918418471275423 -0.9311260670603407 … 0.15429960586669478 1.1944539960204115; 0.0 0.0032977922250339766 … 0.005415281988535716 -0.0033636080585655497], true, 92, 163, 1:1, 1:1, MinAICc(2))

We can get the coefficients matrix for each variable + intercept as usual though now there is a set of coefficients for the model for repetitions and for inclusions

In [18]:
cm <- jl$eval("coef(m)")
coefspos <- cm[1] # repetition
coefszero <- cm[2] # inclusion

In [19]:
coefspos

-4.10095408,-2.46311402,-5.69913212,-5.811141,-4.562263,-5.653145346,-2.8389103,-4.7322153,-3.87792475,-4.441343,⋯,-4.917323,-3.01996764,-3.81827965,-5.19695206,-4.20262645,-3.974637,-5.28172697,-4.68035784,-4.97128452,-3.99263610
-0.02713336,-0.03288065,-0.05020055,0.000000,0.000000,-0.006017206,0.0133696,0.0326247,0.02040813,0.000000,⋯,0.000000,-0.03340693,0.01541684,-0.03889998,0.01138997,0.000000,0.01084449,0.02035161,0.01634997,0.02509239


In [20]:
coefszero

-0.5918418,-0.931126067,0.033016312,-0.37745848,-0.478606926,-0.06758676,3.204239,0.20563006,0.96301244,-0.5918418,⋯,0.331805533,-1.12500898,-0.501823,-0.5918418,-0.243609690,0.72372502,-0.297698794,0.707424066,0.154299606,1.194453996
0.0000000,0.003297792,-0.001667455,0.01815652,0.006536081,-0.01706256,0.000000,-0.01729066,0.00736779,0.0000000,⋯,-0.008679934,0.01034119,0.000000,0.0000000,-0.001827842,-0.02699193,0.004191436,0.008971672,0.005415282,-0.003363608


By default we only return the AICc maximizing coefficients.
To get the coefficients that minimize say the BIC criterion, run

In [21]:
jl$command("paths = hdmrpaths(jcovars, jcounts)")
cm <- jl$eval("coef(paths, MinBIC())")
coefspos <- cm[1] # repetition
coefszero <- cm[2] # inclusion

HDMRPaths{InclusionRepetition{GammaLassoPath{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},LogitLink},NaiveCoordinateDescent{Float64,true,Array{Float64,2},Lasso.RandomCoefficientIterator,Array{Float64,1}}},Float64},GammaLassoPath{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Poisson{Float64},LogLink},NaiveCoordinateDescent{Float64,true,Array{Float64,2},Lasso.RandomCoefficientIterator,Array{Float64,1}}},Float64}},UnitRange{Int64}}(InclusionRepetition{GammaLassoPath{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},LogitLink},NaiveCoordinateDescent{Float64,true,Array{Float64,2},Lasso.RandomCoefficientIterator,Array{Float64,1}}},Float64},GammaLassoPath{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Poisson{Float64},LogLink},NaiveCoordinateDescent{Float64,true,Array{Float64,2},Lasso.RandomCoefficientIterator,Array{Float64,1}}},Float64}}[InclusionRepetition regression

Positive part regularization path (Poisson{Float64}(λ=1.0) with LogLink() link):
Poisson GammaLassoPath (48) solutions for 2 predictors in 218 iterations):
───────────────────────────────────
 λ pct_dev ncoefs
───────────────────────────────────
 [1] 0.543563 2.357e-13 1
 [2] 0.495274 0.0280649 1
 [3] 0.451275 0.0521033 1
 [4] 0.411185 0.0726866 1
 [5] 0.374657 0.0903009 1
 [6] 0.341373 0.105363 1
 [7] 0.311047 0.118229 1
 [8] 0.283414 0.129208 1
 [9] 0.258236 0.138565 1
[10] 0.235295 0.146531 1
[11] 0.214392 0.153302 1
[12] 0.195346 0.159051 1
[13] 0.177992 0.163926 1
[14] 0.16218 0.168054 1
[15] 0.147772 0.171545 1
[16] 0.134645 0.174494 1
[17] 0.122683 0.176982 1
[18] 0.111784 0.179079 1
[19] 0.101854 0.180845 1
[20] 0.0928053 0.182329 1
[21] 0.0845607 0.183577 1
[22] 0.0770486 0.184624 1
[23] 0.0702038 0.185502 1
[24] 0.0639671 0.186237 1
[25] 0.0582844 0.186854 1
[26] 0.0531066 0.187369 1
[27] 0.0483888 0.187801 1
[28] 0.04409 0.188161 1
[29] 0.0401732 0.188462 1
[30] 0.0366043 0.188714 1
[31] 0.0333525 0.188923 1
[32] 0.0303896 0.189098 1
[33] 0.0276898 0.189244 1
[34] 0.0252299 0.189366 1
[35] 0.0229886 0.189467 1
[36] 0.0209463 0.189551 1
[37] 0.0190855 0.189622 1
[38] 0.01739 0.18968 1
[39] 0.0158451 0.189729 1
[40] 0.0144375 0.18977 1
[41] 0.0131549 0.189803 1
[42] 0.0119863 0.189831 1
[43] 0.0109214 0.189855 1
[44] 0.00995121 0.189874 1
[45] 0.00906717 0.18989 1
[46] 0.00826167 0.189904 1
[47] 0.00752772 0.189915 1
[48] 0.00685898 0.189924 1
───────────────────────────────────
Zero part regularization path (Binomial{Float64}(n=1, p=0.5) with LogitLink() link):
Binomial GammaLassoPath (30) solutions for 2 predictors in 127 iterations):
─────────────────────────────────────
 λ pct_dev ncoefs
─────────────────────────────────────
 [1] 0.0315511 2.40918e-14 0
 [2] 0.0287482 0.00192422 1
 [3] 0.0261943 0.00351091 1
 [4] 0.0238673 0.00482049 1
 [5] 0.021747 0.00590223 1
 [6] 0.019815 0.00679638 1
 [7] 0.0180547 0.0075359 1
 [8] 0.0164508 0.00814784 1
 [9] 0.0149893 0.00865443 1
[10] 0.0136577 0.00907395 1
[11] 0.0124444 0.00942149 1
[12] 0.0113389 0.00970947 1
[13] 0.0103316 0.00994815 1
[14] 0.00941374 0.010146 1
[15] 0.00857745 0.0103101 1
[16] 0.00781545 0.0104461 1
[17] 0.00712115 0.010559 1
[18] 0.00648853 0.0106526 1
[19] 0.0059121 0.0107302 1
[20] 0.00538689 0.0107946 1
[21] 0.00490833 0.0108481 1
[22] 0.00447229 0.0108924 1
[23] 0.00407498 0.0109292 1
[24] 0.00371297 0.0109597 1
[25] 0.00338312 0.0109851 1
[26] 0.00308258 0.0110061 1
[27] 0.00280873 0.0110236 1
[28] 0.00255921 0.0110381 1
[29] 0.00233186 0.0110501 1
[30] 0.0021247 0.0110601 1
─────────────────────────────────────
, InclusionRepetition regression

Positive part regularization path (Poisson{Float64}(λ=1.0) with LogLink() link):
Poisson GammaLassoPath (48) solutions for 2 predictors in 226 iterations):
──────────────────────────────────
 λ pct_dev ncoefs
──────────────────────────────────
 [1] 2.07491 0.0 0
 [2] 1.89058 0.0866061 1
 [3] 1.72262 0.152064 1
 [4] 1.56959 0.202331 1
 [5] 1.43015 0.241439 1
 [6] 1.3031 0.272192 1
 [7] 1.1873

In [22]:
coefspos

-4.10095408,-2.46311402,-5.69913212,-5.811141,-4.562263,-5.653145346,-2.8389103,-4.7322153,-3.87792475,-4.441343,⋯,-4.917323,-3.01996764,-3.81827965,-5.19695206,-4.20262645,-3.974637,-5.28172697,-4.68035784,-4.97128452,-3.99263610
-0.02713336,-0.03288065,-0.05020055,0.000000,0.000000,-0.006017206,0.0133696,0.0326247,0.02040813,0.000000,⋯,0.000000,-0.03340693,0.01541684,-0.03889998,0.01138997,0.000000,0.01084449,0.02035161,0.01634997,0.02509239


In [23]:
coefszero

-0.5918418,-0.931126067,0.033016312,-0.37745848,-0.478606926,-0.06758676,3.204239,0.20563006,0.96301244,-0.5918418,⋯,0.331805533,-1.12500898,-0.501823,-0.5918418,-0.243609690,0.72372502,-0.297698794,0.707424066,0.154299606,1.194453996
0.0000000,0.003297792,-0.001667455,0.01815652,0.006536081,-0.01706256,0.000000,-0.01729066,0.00736779,0.0000000,⋯,-0.008679934,0.01034119,0.000000,0.0000000,-0.001827842,-0.02699193,0.004191436,0.008971672,0.005415282,-0.003363608


## Sufficient reduction projection

A sufficient reduction projection summarizes the counts, much like a sufficient
statistic, and is useful for reducing the d dimensional counts in a potentially
much lower dimension matrix `z`.

To get a sufficient reduction projection in direction of `Rem` for the above
example

In [24]:
jl$eval("srproj(m,jcounts,1,1)")

0.028175143,-0.0001159315,30,22
0.016418996,-0.0008058789,28,33
-0.001015670,-0.0059385017,44,34
-0.002358679,-0.0115558569,10,17
-0.047935776,-0.0097930356,17,22
-0.023847399,-0.0040984682,13,20
0.018328712,0.0008254937,5,14
0.011159980,0.0022287515,6,14
0.015826070,0.0016236662,8,19
0.012626627,-0.0069825601,4,17
0.029240021,0.0018497984,21,19


Here, the first column is the SR projection from the model for positive counts, the second is the the SR projection from the model for hurdle crossing (zeros), and the third is the total count for each observation.

## Counts Inverse Regression (CIR)

Counts inverse regression allows us to predict a covariate with the counts and other covariates.
Here we use hdmr for the backward regression and another model for the forward regression.
This can be accomplished with a single command, by fitting a CIR{HDMR,FM} where the forward model is FM <: RegressionModel.

In [25]:
jl$command("using GLM: LinearModel") # loading up the GLM model for the forward regression
jl$command("spec = CIR{HDMR,LinearModel}") # model class specification
jl$command("mf = @model(h ~ President + Rem, c ~ President + Rem)") # model formula
jl$eval("cir = fit(spec, mf, jcovarsdf, jcounts, :Rem, nocounts=true)")

CIR{HDMR,LinearModel}

2-part model: [h ~ President + Rem, c ~ President + Rem]

Julia Object of type TableCountsRegressionModel{CIR{HDMR,LinearModel},DataFrame,SparseMatrixCSC{Float64,Int64}}.
TableCountsRegressionModel{CIR{HDMR,LinearModel},DataFrame,SparseMatrixCSC{Float64,Int64}}

2-part model: [h ~ President + Rem, c ~ President + Rem]

Forward model coefficients for predicting Rem:
─────────────────────────────────────────────────────────────────────────────────────────────────────────
                                         Coef.    Std. Error      t  Pr(>|t|)    Lower 95%      Upper 95%
─────────────────────────────────────────────────────────────────────────────────────────────────────────
(Intercept)                         17.4837       11.3688      1.54    0.1285    -5.17963      40.147
President: Calvin Coolidge          13.0088       13.4688      0.97    0.3374   -13.8407       39.8584
President: Donald J. Trump          -2.14014      11.22       -0.19    0.8493   -24.5069       20.2266
President: Dwight D. Eisenhower    -10.4126        8.31052    -1

where the ```nocounts=True``` means we also fit a benchmark model without counts. 
The last few coefficients are due to text data.
`zpos` is the SR projection summarizing the information in repeated use of terms.
`zzero` is the SR projection summarizing the information in term inclusion.
`m` is the total number of excess counts.
`ℓ` is the total number of nonzero counts.

We can get the forward and backward model coefficients with

In [26]:
jl$eval("coeffwd(cir)")

[1]   17.48368782   13.00884227   -2.14014450  -10.41262276   -6.29262698
 [6]  -22.24412390  -20.26747781   -5.36948700   -9.36384785  -36.12777137
[11]  -14.21891660  -18.40668610  -18.02974937  -33.77949873  -16.73443710
[16]   -3.09841666  477.13098746 9002.65290611   -0.05061284    0.03673466

In [27]:
jl$eval("coefbwd(cir)")

[[1]]
           [,1]      [,2]        [,3]      [,4]      [,5] [,6]         [,7]
 [1,] -7.505779 -3.282723 -5.62136487 -5.652125 -4.648856    0 -2.339185684
 [2,]  0.000000  0.000000  0.00000000  0.000000  0.000000    0 -1.392722095
 [3,]  0.000000  0.000000  0.00000000  0.000000  0.000000    0  0.000000000
 [4,]  0.000000  0.000000  0.00000000  0.000000  0.000000    0 -1.161731770
 [5,]  4.133066  0.000000  0.00000000  0.000000  0.000000    0  0.000000000
 [6,]  0.000000  0.000000  0.00000000  0.000000  0.000000    0  0.384143438
 [7,]  0.000000  1.548887  1.86918130  0.000000  0.000000    0 -0.155145281
 [8,]  0.000000  0.000000  0.00000000  0.000000  1.152582    0  0.319885626
 [9,]  0.000000  0.000000 -1.98595262  0.000000  0.000000    0 -2.622315710
[10,]  0.000000  0.000000  0.00000000  0.000000  0.000000    0 -0.728698645
[11,]  4.261420  0.000000  0.00000000  0.000000  0.000000    0 -1.092640287
[12,]  0.000000  0.000000  0.00000000  0.000000  0.000000    0 -1.687828035
[13,] 

The fitted model can be used to predict vy with new data

In [28]:
jl$assign("jcovarsnewdata", rcovarsdf[1:10,]) # assign new covariates data to julia dataframe

# convert sparse counts matrix for new data
A <- as(rcounts[1:10,], "TsparseMatrix") 
jl$assign("i", A@i + 1)
jl$assign("j", A@j + 1)
jl$assign("v", A@x)
jl$assign("m", dim(A)[1])
jl$assign("n", dim(A)[2])
jl$command("jcountsnewdata = sparse(i, j, v, m, n);")

In [29]:
jl$eval("HurdleDMR.predict(cir, jcovarsnewdata, jcountsnewdata)")

[1]  35.01328  29.84672 -21.60377 -24.86943 -33.69575 -25.10105  11.73583
 [8]  19.33178  17.27555  10.81593

We can also predict only with the other covariates, which in this case
is just a linear regression

In [30]:
jl$eval("HurdleDMR.predict(cir, jcovarsnewdata, jcountsnewdata; nocounts=true)")

[1]  32.43000  32.43000 -26.31750 -26.31750 -26.31750 -26.31750  13.50917
 [8]  13.50917  13.50917  13.50917

Kelly, Manela, and Moreira (2021) show that the differences between DMR and HDMR can be substantial in some cases, especially when the counts data is highly sparse.

Please reference the paper for additional details and example applications.